# **Tutoriel** - Fine-tuning de Llama

<img src="https://voicebot.ai/wp-content/uploads/2023/03/stanford-alpaca.png" alt="Falcon logo"  width="500"/>

Ce carnet de code permet d'effectuer le fine-tuning d'un grand modèle de langue développé par Meta, Llama. Tout en n'étant pas diffusé en open source mais seulement à des fins de recherche non-commercial, Llama a eu un impact significatif sur le développement de LLMs ouverts alternatifs à chatGPT. Une grande partie des méthodes et des jeux de données utilisés pour adapter les LLMs ont été d'abord pensés pour LLama.

LLama est disponible sous quatre versions à 7B, 13B, 33B and 65B : le B correspond ici à un milliard de paramètres. Nous allons ici utiliser la plus grande version compatible avec Google Colab (13B).

Ce carnet de code est basé sur une version modifiée de LLMTune. LLMTune est un projet de recherche de Cornell Tech et de Cornell University (Cornell University). L'exécution de la version originale sur Google Colab donne lieu à plusieurs bugs que j'ai corrigés.

Cette démonstration ne fait tourner qu'une seule *epoch* ce qui est suffisant pour avoir un premier aperçu. Pour obtenir un bon modèle, il est conseillé de faire tourner le fine-tuning pendant trois *epochs*. Sur notre corpus de démonstration de 2000 instructions une *epoch* prendra environ 1h15.

# Installation

En tout premier lieu nous vérifions si nous disposons de suffisamment de mémoire vive (au moins 24go) sinon ce n'est pas la peine de lancer le script.

In [ ]:
!nvidia-smi

Sun Jun 11 14:57:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    40W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Une telle configuration est suffisante pour le fine-tuning de llama-7b-4bit basé sur LLMTune (mais pas pour llama-13b-4bit, sans même parler des versions suivantes)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/llama"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/llama


Nous allons utiliser llmtune. C'est une petite application python disponible sur Github qui permet d'effectuer le fine-tuning de Llama (pour l'instant depuis mon fork comme il y a un bug sur la version de base)

In [ ]:
!git clone https://github.com/Pclanglais/llmtune.git

Cloning into 'llmtune'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 136 (delta 51), reused 36 (delta 29), pack-reused 54
Receiving objects: 100% (136/136), 45.38 KiB | 3.24 MiB/s, done.
Resolving deltas: 100% (62/62), done.


Nous installons également les dépendances.

In [ ]:
%cd llmtune
!pip install -r requirements.txt
!python setup.py install

/content/drive/MyDrive/llama/llmtune
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Using cached transformers-4.28.0-py3-none-any.whl
  Using cached peft-0.3.0.dev0-py3-none-any.whl
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ******************

Enfin nous récupérons le modèle de base de Llama

In [ ]:
!wget https://huggingface.co/kuleshov/llama-13b-4bit/resolve/main/llama-13b-4bit.pt

--2023-07-10 18:05:52--  https://huggingface.co/kuleshov/llama-13b-4bit/resolve/main/llama-13b-4bit.pt
Resolving huggingface.co (huggingface.co)... 13.224.249.43, 13.224.249.10, 13.224.249.44, ...
Connecting to huggingface.co (huggingface.co)|13.224.249.43|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/5d/fe/5dfe252b94712f657ec0467f7dbddf7852ea12f46ffac06e5efd8b771608707b/56956757a1cf930a1881e7ade7bc4cdd243d88517b7d82cb06c01f11463caf38?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-13b-4bit.pt%3B+filename%3D%22llama-13b-4bit.pt%22%3B&Expires=1689271552&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTI3MTU1Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy81ZC9mZS81ZGZlMjUyYjk0NzEyZjY1N2VjMDQ2N2Y3ZGJkZGY3ODUyZWExMmY0NmZmYWMwNmU1ZWZkOGI3NzE2MDg3MDdiLzU2OTU2NzU3YTFjZjkzMGExODgxZTdhZGU3YmM0Y2RkMjQzZDg4NTE3YjdkODJjYjA2YzAxZjExNDYzY2Fm

## Le corpus d'instructions

Nous récupérons un set d'instruction. Dans l'optique d'un simple exemple de démonstration nous utilisons ici une sélection aléatoire de différents corpus d'instructions traduits en français par le projet Vicogne.

In [ ]:
!wget https://raw.githubusercontent.com/opinionscience/EUInstruct/main/sample/french_novel_17_instruction_simple.json
!unzip instruction_french_novel.json.zip

--2023-06-19 14:27:22--  https://raw.githubusercontent.com/opinionscience/EUInstruct/main/translated_sample/instruction_french_novel.json.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15464581 (15M) [application/zip]
Saving to: ‘instruction_french_novel.json.zip’

instruction_french_ 100%[===================>]  14.75M  89.1MB/s    in 0.2s    

2023-06-19 14:27:23 (89.1 MB/s) - ‘instruction_french_novel.json.zip’ saved [15464581/15464581]

Archive:  instruction_french_novel.json.zip
  inflating: instruction_french_novel.json  
  inflating: __MACOSX/._instruction_french_novel.json  


Ces instructions utilisent le format classique du projet Alpaca de Stanford : *instructions*, *input* (optionnellement) et *output*. En résumé, les instructions correspondent à des exemples de prompts que pourraient laisser les utilisateurs du LLM, les *outputs* à la réponse que le LLM devrait générer et les *inputs* apporte des éléments de contextes supplémentaires (par exemple sous la forme de textes cités en exemple)

In [ ]:
import json

with open('french_novel17_instruction.json', 'r') as f:
    data = json.load(f)

json_formatted_str = json.dumps(data[0:3], indent=2)

print(json_formatted_str)

[
  {
    "instruction_id": "gbooks_Yvg5AAAAcAAJ.pdf_141",
    "instruction": "\u00c9crivez un texte qui d\u00e9peint un affrontement verbal entre Lao damas et Eurialus, o\u00f9 ce dernier insulte Ulysse en le traitant de marin incomp\u00e9tent. Le style utilis\u00e9 doit \u00eatre soutenu et empreint d'une certaine gravit\u00e9.",
    "input": "",
    "output": "Si Lao damas fut content de cette mode\u017fte repon\u017feil n'en fut pas de m\u00eame de quelques -uns des autres qui \u017fe per\u017fuadant que s'il \u017fe defen doit d'entrer en licecela ne provenoit que de \u017fa foiblefle & de fon peu d'experience dans ces \u017fortes d'exercices \u017fe mirent \u00e0 le prefler d'u ne maniere fort incivile . Eurialus particuliere ment tout boufi de l'honneur qu'il venoit d'ac querir piqua Uli\u017fle pardes paroles non leule. ment dures mais injurieu\u017feslc taxant d'e\u017ftre plus propre \u00e0 courir la mer \u017fur quelque vailleau CON"
  },
  {
    "instruction_id": "gbooks_jS

# Finetuning du modèle

Tout est prêt à lancer le fine-tuning du modèle. Nous allons juste créer le dossier d'accueil des fichiers de fine-tuning

In [ ]:
!mkdir llama-13b-novel17

mkdir: cannot create directory ‘llama-7b-french-novel’: File exists


Et nous sommes prêt à lancer la grande commande. Il y a beaucoup de paramètre mais seulement quelqu'uns sont importants :
* Nous allons utiliser le modèle Llama-7b de base et leurs poids correspondants (llama-7b-4bit)
* Le fine-tuning sera effectué sur le set d'instruction *eu_translate_fr_sample.json* (évidemment à changer si vous optez pour un autre jeu de données).
* Les fichiers du modèle seront placés dans le dossier *llama-7b-sample* (de nouveau à changer pour le nom de votre modèle).
* Nous ne ferons tourner le fine-tuning que sur une *epoch* ce qui est suffisant pour un premier test.

Après avoir lancé le script, Google Colab va tourner pendant un peu moins de 40 minutes.

Si tout se passe bien vous verrez défiler le processus d'entraînement avec trois indicateurs régulièrement réactualisés : "{'loss': 1.8581, 'learning_rate': 0.0002993736951983298, 'epoch': 0.0}" :
* Le "loss" c'est en quelque sorte le taux d'erreur du modèle : plus cette mesure est basse et plus le modèle parvient à prédire des textes assez approchants de ceux qui sont présent dans le corpus d'instruction.
* Le *learning rate* (taux d'apprentissage) c'est la capacité du modèle à mémoriser de nouveaux éléments mais aussi à en oublier des anciens. Cet indicateur va constamment baisser au fur et à mesure de l'apprentissage.
* L'*epoch* c'est le cycle d'apprentissage. Comme nous n'avons défini qu'une *epoch* cela correspondra à des pourcentages (de 0 à 0.99 à la fin de l'entraînement).

In [ ]:
!llmtune finetune \
    --model llama-13b-4bit \
    --lr=3e-4 \
    --epochs 1 \
    --save_total_limit 3 \
    --save_steps 500 \
    --weights llama-13b-4bit.pt \
    --adapter llama-13b-novel17 \
    --mbatch_size=1 \
    --batch_size=2 \
    --epochs=1 \
    --cutoff_len=256 \
    --lora_r=8 \
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --warmup_steps=5 \
    --dataset french_novel17_instruction2.json

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
trainable params: 6553600 || all params: 334648320 || trainable%: 1.958354370343171
Extracting data files: 100% 1/1 [00:01<00:00,  1.95s/it]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b67ee2f1b68744d9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.
100% 1/1 [00:00<00:00, 789.74it/s]
2023-07-10 19:20:15.761993: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating you

In [ ]:
!ls

build				   llama-7b-french-novel  README.md
dist				   llmtune		  requirements.txt
instruction_french_novel.json	   llmtune.egg-info	  setup.py
instruction_french_novel.json.zip  __MACOSX
llama-7b-4bit.pt		   quant_cuda.egg-info


Si tout se passe bien, vous devrez avoir comlètement fini l'entraînement.

Après un petit temps de synchronisation entre Google Colab, vous allez voir apparaître deux fichiers dans le dossier du modèle de fine-tuning : adapter_model.bin (le modèle proprepement dit) et adapter_model.config (un fichier de configuration). À noter que le modèle de fine-tuning est considérablement plus petit que le modèle d'origine : c'est en quelque sorte un modèle complémentaire qui vient ajuster le LLM (et il en aura toujours besoin pour fonctionner).

# Générer du texte

Et maintenant il est possible de générer du texte. La fonction par défaut de llmtune n'est pas pour l'instant pas très pratique mais cela devrait s'améliorer prochainement.

In [ ]:
!llmtune generate \
    --model llama-13b-4bit \
    --weights llama-13b-4bit.pt \
    --adapter llama-13b-novel17 \
    --max-length 400 \
    --min-length 200 \
    --instruction "Écrivez un texte littéraire dans le style du XVIIe siècle décrivan un peuple de lamas intelligents au Pérou. Vous pouvez notamment évoquer les mœurs et coutumes de ces animaux très savants. ###OUTPUT"

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
llama-13b-novel17 loaded
2023-07-10 20:06:42.312111: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Les lamas dont l'on vient de faire mention sont de ce grand peuple ,ils entretiennent l'élection à leur cheit & ſont ſentimentaux & galans comme deſſus. Ces animaux ont le pied palmatif,& ont les pattes couvertes de vaiſſes poils blancs. Ils entretiennent par l'organe pariétal des eaux du corps en fournissant des uſages de chirurgie. Ils prètent vn peu de mal aux lamas pour leur ravir le tire ou leur couper la jambe. Pour peu qu'on fcure 10 cacher le pain de la faiſon à ſes petits,ils ne les lairont pas . Il ſeroient trop heureux ſi on les laiſſoit a</s><s>
